In [ ]:
### import libraries

# format
import pandas as pd

# plot
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D


# math
import numpy as np
import math
from scipy.linalg import inv, det
from numpy.linalg import eig

from scipy import stats

#PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

## Introduction
#### Authors: Laura Silvana Alvarez, Florencia Maite Luque and Simon Schmetz
The following project documentation was written as work assignment for the module "Multivariate Analysis" of the Master in Statistics for Data Science at the Universidad Carlos III de Madrid. It contains the Multivariate Analysis of a Kaggel dataset on Sleep Health and Lifestyle (https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset). The work is split into two parts, where in a first part a exploratory data analysis is performed, some data preprocessing steps are taken and a Prinicipal Component Analysis (PCA) is performed. In the second part, distance based metrics will then be applied to identify clusters emerging from the PCA. 

In [ ]:
# load data
data_raw = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")

 The dataset at hand is composed out of the collumns shown in the table below. It has been modified compared to the kaggle source data by turning the "Sleep Disorder" Variable into a binary variable (yes/no) and by seperating the blood pressure variables into the two variables blood pressure systolic and blood pressure diastolic. The resulting data set is composed out of the variables shown in the following table. 

| **Variable**                    | **Description**                                                                                              |
|---------------------------------|--------------------------------------------------------------------------------------------------------------|
| Person ID                       | An identifier for each individual.                                                                           |
| Gender                          | The gender of the person (Male/Female).                                                                      |
| Age                             | The age of the person in years.                                                                              |
| Occupation                      | The occupation or profession of the person.                                                                  |
| Sleep Duration (hours)          | The number of hours the person sleeps per day.                                                               |
| Quality of Sleep (scale: 1-10)  | A subjective rating of the quality of sleep, ranging from 1 to 10.                                          |
| Physical Activity Level (minutes/day) | The number of minutes the person engages in physical activity daily.                               |
| Stress Level (scale: 1-10)      | A subjective rating of the stress level experienced by the person, ranging from 1 to 10.                    |
| BMI Category                    | The BMI category of the person (e.g., Underweight, Normal, Overweight).                                      |
| Blood Pressure (systolic) | The blood pressure measurement of the person (systolic pressure)|
| Blood Pressure (diastolic) |  The blood pressure measurement of the person (diastolic pressure)|
| Heart Rate (bpm)                | The resting heart rate of the person in beats per minute.                                                    |
| Daily Steps                     | The number of steps the person takes per day.                                                                |
| Sleep Disorder                  | The presence or absence of a sleep disorder in the person (Binary)|

In [ ]:
### Preprocess Data
data = data_raw.copy()

# rename columns for easier use
rename_dict = {
    'Person ID':'person_id',
    'Gender': 'gender',
    'Age':'age',
    'Occupation':'occupation',
    'Sleep Duration':'sleep_duration',
    'Quality of Sleep':'quality_of_sleep',
    'Physical Activity Level':'physical_activity_level',
    'Stress Level':'stress_level',
    'BMI Category':'bmi_category', 
    'Blood Pressure':'blood_pressure', 
    'Heart Rate':'heart_rate', 
    'Daily Steps':'daily_steps',
    'Sleep Disorder':'sleep_disorder' 
}
data.rename(columns=rename_dict, inplace=True)

# change dtype
data['quality_of_sleep'] = data['quality_of_sleep'].astype(str)
data['stress_level'] = data['stress_level'].astype(str)

# make sleep disorder binary
data['sleep_disorder'] = data['sleep_disorder'].map(lambda x: '1' if x in ['Insomnia','Sleep Apnea'] else '0').astype(str)

# split blood pressure into diastolic & systolic
data[["blood_pressure_systolic","blood_pressure_diastolic"]] = data["blood_pressure"].str.split('/',expand=True)
data["blood_pressure_diastolic"] = pd.to_numeric(data['blood_pressure_diastolic'])
data["blood_pressure_systolic"] = pd.to_numeric(data["blood_pressure_systolic"])


#data.info()

Based on unique value counts and the variable description, the numeric and categorical variables are identified as: 

| **Type**              | **Variables**                                                       |
|------------------------|---------------------------------------------------------------------|
| **Numeric Variables**  | age, sleep_duration, physical_activity_level, heart_rate, daily_steps, blood_pressure_systolic, blood_pressure_diastolic |
| **Categorical Variables** | gender, occupation, quality_of_sleep, stress_level, bmi_category, sleep_disorder |




In [ ]:
# # set up column lists
numeric_variables = ['age','sleep_duration','physical_activity_level','heart_rate','daily_steps','blood_pressure_systolic','blood_pressure_diastolic']
categorical_variables = ['gender','occupation','quality_of_sleep','stress_level','bmi_category','sleep_disorder']
# data.nunique()

In [ ]:
#data[categorical_variables].apply(pd.unique)

Reviewing the unique values of the identified categorical values, a duplicate in bmi_category in the values "Normal" and "Normal Weight" can be identified. This is solved by replacing all instances of "Normal Weight" with "Normal"

In [ ]:
data['bmi_category'] = data['bmi_category'].replace('Normal Weight', 'Normal')
#data[["bmi_category"]].apply(pd.unique)


Doing so concludes the required initial preprocessing steps and allows to begin with the first part of this project work.

### Part 1 - Exploratory Analysis and Dimension Reduction via PCA
The first part of this work contains the initial exploratory analysis of the dataset as well as a Principal Component Analysis (PCA) of the dataset.

### Exploratory Data Analysis

The Exploratory Data Analysis contains a general overview of the datasets structure and correlations.

In [ ]:
# show first rows of dataframe
#data.head()

The categorical Variables can be further sperated into the following categories with: 

Two Binary variable:
- gender
- sleep disorder

Three ordinal variables: 
- quality of sleep
- stress level
- bmi_category

And one nominal variables:
- occupation

Plotting barplots for the categorical variables, beginning with the binary variables of gender and sleep disorder, shows an even distribution between male and female and a fairly even distribution between observations with and without sleep disorders. In the meantime, stress level shows a decrease in observations towards higher stress levels, and equally, quality of sleep and body mass index (BMI) show a continuous decrease in observations for quality of sleep from 8 to 4 and for BMI categories from normal to obese. Lastly, the variable occupation shows a somewhat uneven distribution of observations between the different occupations, with the two occupations that individually contribute the most to the overall dataset being Nurses and Doctors. The bias of potential overrepresentation of the medical field with irregular working hours in shifts cannot be further analyzed in this work but has to be taken into account in later conclusions.

In [ ]:
# Barplots for categorical
def pie_bar(data, features):
    n_features = len(features)
    n_cols = 3 
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(10, 3 * n_rows))
    axs = axs.flatten()
    
    for idx, feature in enumerate(features):
        count_values = data[feature].value_counts()
        axs[idx].bar(x=count_values.index, height=count_values.values, edgecolor='black')
        axs[idx].set_xticks(range(len(count_values)))
        axs[idx].set_xticklabels(count_values.index, rotation=45, ha='right')
        axs[idx].set_title(f"Bar Plot for {feature}")
    
    plt.tight_layout()
    plt.show()

pie_bar(data, categorical_variables)


Of the seven numerical variables, all are continuous. Plotting histograms and boxplots side by side, two categories emerge:

- Measured variables
- Estimated/rounded variables (variables that were probably estimated as part of a questionnaire by participants)

For the measured variables, age, heart rate, and sleep duration, a more or less continuous distribution can be identified. Age, while showing variance, still is somewhat uniformly distributed between the limits of around 30 to 60. Sleep duration appears to show three groups: one group having very little sleep (up to 6.5 hours) and one group sleeping for longer times (more than 8 hours), with a dip in sleeping time observed between these two groups. As the last variable of this group, the heart rate is shown to have the majority of its observations between 65 and 75, with visual hints of a normal distribution. However, it shows significant outliers for higher heart rates in both the histogram and boxplot, which will need to be addressed in the preprocessing step.
The estimated/rounded variables (physical activity level, daily steps, blood pressure systolic/diastolic) show a specific characteristic with oscillations between highs and lows, attributed to the way people estimate numeric values in increments, like evaluating physical activity level (mins/day) mostly in increments of 15 minutes (half an hour, 45 minutes, one hour, etc.). The lows in between then show observations with more specific answers like 42 minutes, leading to the somewhat irregular appearance of the histograms. Taking this into account, the physical activity level shows a fairly uniform distribution of observations, while daily steps tend more toward a right-skewed uniform distribution, indicating an overall potentially above-average fit sample of people. The high percentage of medical workers with great walking distances as part of their profession might further contribute to this. Meanwhile, the blood pressure systolic/diastolic appears to be more or less evenly distributed.

In [ ]:
### Histograms + Boxplot
def hist_box_plot(data, numeric_variables):
    fig, axs = plt.subplots(2, len(numeric_variables), figsize=(len(numeric_variables)*5, 5))
    for i, feature in enumerate(numeric_variables):
        
        # Plot histogram
        axs[0, i].hist(data[feature], bins=10, density=True, alpha=0.6, color='b', edgecolor="black")
        axs[0, i].set_title(f"Hist of {feature}")

        # Plot boxplot
        axs[1, i].boxplot(data[feature], vert=False, patch_artist=True, medianprops=dict(color="black"))
        axs[1, i].set_title(f"Boxplot of {feature}")
    
    plt.tight_layout(pad=2.0)

hist_box_plot(data, numeric_variables[0:3])
hist_box_plot(data, numeric_variables[3:5])
hist_box_plot(data, numeric_variables[5:])

As a first attempt to evaluate the correlations found in this dataset, the following set of Metrics is applied and plotted.


\begin{aligned}
q_1 &= \left(1 - \frac{\min \lambda_j}{\max \lambda_j} \right)^{p+2}, & \quad q_4 &= \left(\frac{\max \lambda_j}{p} \right)^{3/2}, \\
q_2 &= 1 - \frac{p}{\sum_{j=1}^p \left(1/\lambda_j\right)}, & \quad q_5 &= \left(1 - \frac{\min \lambda_j}{p} \right)^5, \\
q_3 &= 1 - \sqrt{|R|}, & \quad q_6 &= \sum_{j=1}^p \frac{1 - 1/r_{ij}}{p}.
\end{aligned}



In [ ]:
### Statistical Analysis & intercorrelation

means = data[numeric_variables].mean()
variances = data[numeric_variables].var()
df_covariance_matrix = data[numeric_variables].cov()
df_correlation_matrix = data[numeric_variables].corr()

def intercorrelations(X,categorical_filter = "whole Dataset",label = None):
    n, p = X.shape
    R = np.corrcoef(X, rowvar=False)
    lambda_vals, _ = eig(R)
    rjj = np.diag(inv(R))
    q = np.zeros(6)
    q[0] = (1 - min(lambda_vals) / max(lambda_vals)) ** (p + 2)
    q[1] = 1 - p / np.sum(1. / lambda_vals)
    q[2] = 1 - np.sqrt(det(R))
    q[3] = (max(lambda_vals) / p) ** (3 / 2)
    q[4] = (1 - min(lambda_vals) / p) ** 5
    q[5] = np.sum((1 - 1. / rjj) / p)
    
    # plot
    plt.plot(range(1, 7), q, marker='o', linestyle='-', label=label)
    plt.xlabel("intercorrelation metric")
    plt.ylabel("intercorrelation score")
    plt.suptitle("Intercorrelation Metric for filter: "+categorical_filter)
    plt.legend()

    return None

plt.figure(figsize=(5,2.5))
intercorrelations(data[numeric_variables])

Applying the same method to a dataset filtered on the binary variable sleep disorder shows an overall higher than before correlation in the subset with sleep disorder while the subset for people without sleep disorder shows lower correlation metrics than the joined dataset. 

In [ ]:
#intercorrelation by category function
def inter_by_category(df,cat):
    count_values = df[cat].value_counts().index
    plt.figure(figsize=(5,2.5))
    for i in count_values:
        aux_data = df[df[cat]==i]
        intercorrelations(aux_data[numeric_variables],categorical_filter=cat,label=i)

inter_by_category(data,'sleep_disorder')

Expanding the correlation analysis with a pair plot shows a variety of potential correlations between variables, the most notable being the linear correlation between blood pressure systolic and diastolic, as well as between daily steps and heart rate/blood pressure. Further correlation can be seen between physical activity level and sleep duration, while the plot of age vs. sleep duration appears to show certain clusters that may be further analyzed in the second part of this work.

In [ ]:
# pairplot
sns.set(rc={'figure.figsize':(6, 3)})
sns.pairplot(data[numeric_variables], diag_kind='kde', corner=True)
plt.show()

Plotting the correlation corresponding matrix reflects some of the observations made in the pairplot, with the verious near 100% correlation between blood pressure systolic and diastolic very aparent. Two previously less apparent correlations are the ones between age and blood pressure as well as the correlation between physical activity and daily steps. 

In [ ]:
corr_mat = data[numeric_variables].corr()
plt.figure(figsize=(4, 3))  # Adjust the figure size as needed
sns.heatmap(corr_mat, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, square=True, linewidths=0.5)
plt.title('Correlation Plot')
plt.show()

Plotting the covariance matrix however shows an issue with the current format of the data, where daily steps outweighs all other variances due to its scale (3000 - 10000). This issue will be adressed in the next section of part one of this work. 

In [ ]:
def cov_plot(data,numeric_variables):
    corr_mat = data[numeric_variables].cov()
    plt.figure(figsize=(4, 3))  # Adjust the figure size as needed
    sns.heatmap(corr_mat, annot=True, cmap='coolwarm', square=True, linewidths=0.5)
    plt.title('Covariance Matrix')
    plt.show()

cov_plot(data,numeric_variables)

Having a general overview of structure and correlation in the data, the next step is to scaling and outlier issues in the next subsection.

In [ ]:
# # Function for plotting of conditional histograms 
# def plot_categorical_hist(data,
#                           ncols,
#                           numeric_variables,
#                           categorical_variables,
#                           host_stat='count',
#                           figsize=(12, 10)
#                           ):
    
#     nrows = math.ceil(len(numeric_variables)/ncols)
#     fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
#     for i, col in enumerate(numeric_variables):
#         row = i // 2 
#         col_pos = i % 2 
#         sns.histplot(data=data, x=col, bins=10,hue=categorical_variables, kde=True, ax=axes[row, col_pos],stat=host_stat)
#         axes[row, col_pos].set_title(f'Distribution of {col}')

#     # Adjust layout for better spacing
#     plt.tight_layout()
#     plt.show()

# # TODO: Select relevant ones 
# for categorical_variable in categorical_variables:
#     print(categorical_variable)
#     plot_categorical_hist(data=data,
#                           ncols=2,
#                           numeric_variables=numeric_variables,
#                           categorical_variables=categorical_variable,
#                           host_stat='probability',
#                           figsize=(12, 20))

### Preprocessing

The following two issues in the current data set: 
- Outliers in variable "heart rate"
- Scaling issue to (among others) variable "daily steps"

This section corrects outliers, validates skewness and standardizes the numeric variables.

### Outliers and Skewness

The aim of this part of the preprocessing, is to obtain symmetric variables without outliers in order to apply in a correct form the PCA. 

It is observed that only one variable has outliers and positive skewness problems (heart rate). Therefore, the first step is to cut the outliers (4% of the dataframe), and then, check if the skewness problem is also corrected.

In [ ]:
# print skewness
for i in numeric_variables:
    aux_skew = stats.skew(data[i])
    print(f"Skewness of {i} : {aux_skew}")

In [ ]:
summary_stats = []

for i in numeric_variables:
    # Compute the descriptive statistics using scipy.stats.describe
    description = stats.describe(data[i])
    p90 = np.percentile(data[i], 90)
    p95 = np.percentile(data[i], 95)
    p99 = np.percentile(data[i], 99)

    # Store the results as a dictionary (with variable name as key)
    summary_stats.append({
        'Variable': i,
        'Count': description.nobs,
        'Min': description.minmax[0],
        'Mean': description.mean,
        'Percentile 90%': p90,
        'Percentile 95%': p95,
        'Percentile 99%': p99,
        'Max': description.minmax[1],
        'Variance': description.variance
    })

# Convert the list of dictionaries to a DataFrame for easier display
summary_df = pd.DataFrame(summary_stats)

# Display the summary statistics table
#print(summary_df)


In [ ]:
for i in numeric_variables:
    q1 = np.percentile(data[i], 25)
    q3 = np.percentile(data[i], 75)
    RIC = q3 - q1 
    nout = np.sum(data[i] > (q3 + 1.5*RIC))
    print(f"The threshold for {i} upper outliers is  {q3 + 1.5*RIC}")
    print(f" then there are {nout} outliers in this variable, representing the {np.round(nout/374*100,2)} % of the dataset")

We can see that the skewness was also corrected by cutting the oultiers observations. For that reason, there is not needed another type of transformation.

In [ ]:
# temporary data while we have all the transformations
data_cut = data[data['heart_rate'] < 78] 

# CHECKING SKEWNESS AFTER CUTTING OUTLIERS
aux_skew = stats.skew(data_cut["heart_rate"])
print(f"Skewness of heart_rate : {aux_skew}")


# TODO: Add x axis label, turn into two big plots

# function for histogram + boxplots on numerical variables
def hist_box_plot_basic(data, feature):
    fig, axs = plt.subplots(1,2, figsize=(5, 2.5))
    axs[0].hist(data[feature],bins=10,density=True,alpha=0.6,color='b',edgecolor="black")
    axs[0].set_title(f"Hist of {feature}")
    axs[1].boxplot(data[feature],vert=False, patch_artist=True,medianprops=dict(color="black"))
    axs[1].set_title(f"Boxplot of {feature}")
    plt.tight_layout(pad=2.0)


# plot hist % boxplot before and after
hist_box_plot_basic(data,"heart_rate")
hist_box_plot_basic(data_cut,"heart_rate")

### Standardize numeric variables

Having seen in the exploratory data analysis that there exists a strong imbalance in scale between numerical variables in the dataset, the dataset is standardized in this step to mean 0 and scaled on its standarddeviation.

In [ ]:
# scale
data_standardized_numeric = (data_cut[numeric_variables] - data_cut[numeric_variables].mean()) / data_cut[numeric_variables].std()
data_standardized = pd.concat([data_standardized_numeric,data_cut[categorical_variables]], axis =1)


Comparing the boxplots pre-standardized and post-standardized shows the major impact the rescaling has, where daily steps previously dominated and now an even distribution for all numeric variables can be seen.

In [ ]:
# plot
fig, axs = plt.subplots(1,2, figsize=(5, 2.5))
axs[0].boxplot(data[numeric_variables],vert=True, patch_artist=True,medianprops=dict(color="black"))
axs[0].set_xticklabels(numeric_variables, rotation=45, ha="right")
axs[1].boxplot(data_standardized_numeric[numeric_variables],vert=True, patch_artist=True,medianprops=dict(color="black"))
axs[1].set_xticklabels(numeric_variables, rotation=45, ha="right")
fig.suptitle("Boxplots not-standardized vs standardized", fontsize=16)
print()

As a result from scaling, now the covariance matrix can be constructed, showing similar results compared to the previously analyzed correlation matrix. 

In [ ]:
cov_plot(data_standardized,numeric_variables)

### PCA

Having analyzed the data and its characteristic and highly correlated variables identified, as well as having eliminated outliers as well as having standardized the numeric variables, principal component analysis can now be applied in an attempt to reduce dimensionality. 

### Number of Principal Components

By analysing the Explained Variance (eigenvalues) trend, and the Joliffe´s and Kaiser´s criterion, for this project there are selected 3 Principal components that explain the 90\% of the variability.

In [ ]:
data_pca = data_standardized[numeric_variables]
#data_pca.to_csv('C:/Users/Usuario/Desktop/UC3M/SC2/MULTIVARIADO/PROYECTO/data_estandarizada.csv', index=False)
pca = PCA()
pca.fit(data_pca)
print()

In [ ]:
eig_val = pca.explained_variance_
#print("Explained Variance: ", eig_val)

sv = pca.singular_values_
#print("Singular Values: ", sv)

#----------------------- JUST CHECKING MATHS -----------------------
singular_values_squared = sv ** 2
# Compare singular values squared vs eigenvalues
#normalized_singular_values_squared = singular_values_squared / (X_scaled.shape[0] - 1)
#print("Eigenvalues (Explained Variance):", eig_val)
#print("Singular Values Squared (Normalized):", normalized_singular_values_squared)
#--------------------------------------------------------------------------------

# Covariance Matrix
S = np.cov(data_pca, rowvar=False)
print("trace: ", S.trace())

# Kaiser´s criterion
mean_eig = S.trace()/7

# Jollife´s criterion
joliffe = 0.7 * S.trace()/7



In [ ]:

plt.figure(figsize=(5, 2.5))
plt.plot(eig_val, marker = 'o', linestyle = '-', color = 'g')
plt.axhline(y=mean_eig, color='r', linestyle='--', label='Keisers Criterion')
plt.axhline(y=joliffe, color='r', linestyle='-', label='Jollife Criterion')
plt.title("Explained Variance")
plt.xlabel("Principal Component")
plt.ylabel("Eigen Values")

# Change xlabels
tick_positions = [0, 1, 2, 3, 4, 5, 6]
custom_labels = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7']  

# Set custom x-ticks with custom labels
plt.xticks(ticks=tick_positions, labels=custom_labels)

plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# I AM STILL CHECKING THIS ONE, ONLY TRUST 100% THE ONE THAT IS ABOVE


# Explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Show the cumulative explained variance
plt.figure(figsize=(5, 2.5))
plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', color='g')
plt.axvline(x=3, color='r', linestyle='--', label='')
plt.title("Cumulative Explained Variance")
plt.xlabel("Principal Component Index")
plt.ylabel("Cumulative Explained Variance Ratio")

# Adding labels next to the points
for i, (x, y) in enumerate(zip(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance)):
    plt.text(x, y, f'{y:.2f}', horizontalalignment='right', verticalalignment='bottom', fontsize=10)

plt.grid(True)
plt.show()


In [ ]:
# calc delta variance
delta_explained_variance  = []
i = 0
while i < len(cumulative_explained_variance)-1:
    delta_explained_variance.append(cumulative_explained_variance[i+1]-cumulative_explained_variance[i])
    i = i+1
delta_explained_variance.insert(0, cumulative_explained_variance[0])

|    **Value**      |  **PC1**   |  **PC2**   |  **PC3**  |  **PC4**   |  **PC5**   |  **PC6**   |  **PC7**  |
|-------------------|------------|------------|-----------|------------|------------|------------|-----------|
| **Eigen Values**  | 2.98968 | 1.91745 | 1.40832 | 0.35319 | 0.21651 | 0.09865 | 0.01619 |
| **% Variability** | 42.7097 | 27.3922 | 20.1188 | 5.04559 | 3.09299 | 1.40933 | 0.23132 |
| **Cum. Variance** | 42.7097 | 70.1019 | 90.2207 | 95.2663 | 98.3593 | 99.7686 | 100.000 |


In [ ]:
# generate transformed Data Set
pca_transformed = pca.transform(data_pca)
df_pca_transformed =pd.DataFrame(pca_transformed,columns=["PC1","PC2","PC3","PC4","PC5","PC6","PC7"])
df_pca_merged = pd.concat([df_pca_transformed, data_cut[categorical_variables].reset_index()],axis=1)


### Contribution of Variables to Principal Components and Interpretation

Plotting the Composition of the Principal Components as a Barplot shows a general overlap in the contribution of all variables to the new found principle components, making a very clear seperation into characteristics that are described by the prinicipal components a more dificult task. Since all outliers and scale differences have been treated in the preprocessing steps, the dataset simply seems not to have directly obvious meaningfull interpretation of its principal components. While interpreability in more general terms is one of the challenges associated to PCA, an attempt is made to give some intuition to the three principal component dimensions selected to represent this data set. The first principal component PC1 can be interpreted as overall characteristics of a person, with all variables except for sleep_duration somewhat evenly related. PC2 could be interpreted as the dimension of age, with age, leep duration and heart rate mainly contributing. Simelarly, PC3 could be interpreted as the physical condition dimension, with physical activity level, daily steps and sleep duration primarily contributing.

In [ ]:
def eigsort(S):
    # Eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eig(S)
    # Sort Eigenvalues
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    Lambda = sorted_eigenvalues
    
    return sorted_eigenvectors, Lambda

def contribution_plot(ax, arrays, colors, labels, x_labels, width=0.2):
    n_arrays = len(arrays)
    x_indices = np.arange(len(arrays[0])) + 1

    # Barplots
    for i, (array, color, label) in enumerate(zip(arrays, colors, labels)):
        bar_positions = x_indices + (i - n_arrays / 2) * width
        ax.bar(bar_positions, array, width=width, color=color, label=label)

    # Cosmetics
    ax.set_xlabel("$X_n$ PCA Weights (computed from S)")
    ax.set_ylabel("Weight")
    ax.set_title("PC Composition")
    ax.axhline(0, color='black', linewidth=0.8, linestyle='-') 
    ax.set_xticks(x_indices)
    ax.set_xticklabels(x_labels, rotation=30, ha="right")
    ax.legend()

[T, Lambda] = eigsort(S)

# Correlation Matrix
correlation_matrix = pd.DataFrame(
    {col2: [data_pca[col1].corr(df_pca_merged[col2]) for col1 in numeric_variables] for col2 in ["PC1","PC2","PC3"]},
    index=data_pca.columns
)

# Create Figure
fig, axs = plt.subplots(1, 2, figsize=(16, 6),gridspec_kw={'width_ratios': [2.5, 1]})

# Brplot (Contribution)
contribution_plot(
    ax=axs[0],
    arrays=[T[:, 0], T[:, 1], T[:, 2]],
    colors=["royalblue", "indianred", "seagreen"],
    labels=["PC1 Weight", "PC2 Weight", "PC3 Weight"],
    x_labels=numeric_variables,
    width=0.25
)

# Correlation Plot
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", cbar=True, ax=axs[1])
axs[1].set_title("Correlation Matrix: Numeric Columns vs PC")

plt.tight_layout()
plt.show()


Plotting the data transformed into the three first principal components shows a similarly entangled image image as already shown in the barplots, with no obious interpretation. 

In [ ]:

def scatterplot_pc(df, x_col, y_col, hue_col=None, ax=None, expl_var=None):
    # plot
    sns.scatterplot(data=df, 
                    x=x_col, 
                    y=y_col, 
                    hue=hue_col,
                    palette='Set1',
                    ax=ax)
    
    # cosmetics
    ax.set_title(f'Scatter Plot of {y_col} vs {x_col} by \n filter: {hue_col}, joined explaind variance {expl_var} %')
    ax.set_xlabel(x_col)
    ax.set_ylabel(y_col)
    ax.axhline(0)
    ax.axvline(0)


fig, axs = plt.subplots(1, 3, figsize=(12, 4))
scatterplot_pc(df_pca_merged,x_col="PC1",y_col = "PC2", ax = axs[0],expl_var = round(delta_explained_variance[0]+delta_explained_variance[1],3)*100)
scatterplot_pc(df_pca_merged,x_col="PC1",y_col = "PC3", ax = axs[1],expl_var = round(delta_explained_variance[0]+delta_explained_variance[2],3)*100)
scatterplot_pc(df_pca_merged,x_col="PC2",y_col = "PC3", ax = axs[2],expl_var = round(delta_explained_variance[1]+delta_explained_variance[2],3)*100)
plt.show()


By plotting the 3 principal planes obtained labeled with the categorical variables, interesting relations were found. This relations could help us to give names to each PC.

- **PC1 (Wellness):** Can be interpreted as a person wellness because it makes a good separation between having or not a sleep disorder and also of having or not overweight.

- **PC2 (Stress Level):** Higher values of the PC2 indicates higher stress levels (The stress level is well organized in this component). Also, positive values of this component are related with a greater age and having high pressure, while lower values are related with a more active life.

- **PC3 (Physical Activity):** In this component a relation is less clear  with any of the categorical variables, so the interpretation from the continous variables contribution is mantained as the PC name.

In [ ]:
for categorical_variable in categorical_variables:
    fig, axs = plt.subplots(1, 4, figsize=(15, 4), gridspec_kw={'width_ratios': [1, 1, 1, 0.2]})
    
    # Create the scatter plots
    scatter1 = scatterplot_pc(
        df_pca_merged.sort_values(by=[categorical_variable]),
        x_col="PC1",
        y_col="PC2",
        hue_col=categorical_variable,
        ax=axs[0],
        expl_var=round(delta_explained_variance[0] + delta_explained_variance[1], 3)*100
    )
    axs[0].legend().set_visible(False)

    
    scatterplot_pc(
        df_pca_merged.sort_values(by=[categorical_variable]),
        x_col="PC1",
        y_col="PC3",
        hue_col=categorical_variable,
        ax=axs[1],
        expl_var=round(delta_explained_variance[0] + delta_explained_variance[2], 3)*100
    )
    axs[1].legend().set_visible(False) 

    
    scatterplot_pc(
        df_pca_merged.sort_values(by=[categorical_variable]),
        x_col="PC2",
        y_col="PC3",
        hue_col=categorical_variable,
        ax=axs[2],
        expl_var=round(delta_explained_variance[1] + delta_explained_variance[2], 3)*100
    )
    axs[2].legend().set_visible(False)

    
    # Add Manual Legend to side
    handles, labels = axs[0].get_legend_handles_labels()
    axs[3].legend(handles=handles, labels=labels, loc='center', frameon=False)
    axs[3].axis('off') 

    # plot
    plt.tight_layout()
    plt.show()


A first glance at possible meaning behind the principal components is only found when plotting the transformed data in three dimensions with a colorcoding for the categorical features. Doing so, first patterns emerge. Some observable pattern include: 
- male data points being closer to the origin and female values data points
- clusters of occupations visible, sometimes in combination with stress level like two clusters for nurses, one with low and one with high stress level

In [ ]:
def scatterplot_3d(ax, df, x_col, y_col, z_col, hue_col):
    
    unique_hues = df[hue_col].unique()
    for hue in unique_hues:
        hue_df = df[df[hue_col] == hue]
        ax.scatter(hue_df[x_col], hue_df[y_col], hue_df[z_col], label=hue)

    # cosmetics
    ax.set_xlabel(x_col)
    ax.set_ylabel(y_col)
    ax.set_zlabel(z_col)
    ax.set_title(f'PC 1-3 with hue: {hue_col}')
    ax.legend(title=hue_col)

def scatterplot_3d_subplots(df, categorical_variables, x_col, y_col, z_col):
    
    n_cols = 3
    n_rows = (len(categorical_variables) + 1) // n_cols
    fig = plt.figure(figsize=(15,10))

    # make subplots    
    for i, categorical_variable in enumerate(categorical_variables):
        ax = fig.add_subplot(n_rows, n_cols, i + 1, projection='3d')
        scatterplot_3d(ax, df.sort_values(by=[categorical_variable]), x_col, y_col, z_col, categorical_variable)
    
    # cosmetics
    plt.tight_layout()
    plt.show()


# Call the function to create subplots
scatterplot_3d_subplots(df_pca_merged, categorical_variables, x_col="PC1", y_col="PC2", z_col="PC3")


These emerging patterns between the principal components, the numerical and categorical variables and more specifically sleep disorder show an apparent clustering potential that can be further investigated in the second part of this project via distance based metrics. 

### Stability of the Principal components

In [ ]:
def PCAcovstability(X):
    #create H matrix to do covariance for pca
    n, p = X.shape
    H = np.eye(n) - np.ones((n, n)) / n
    X = np.dot(H, X)
    S = np.cov(X, rowvar=False, bias=True) 
    
    T, D = eigsort(S)
    Y = np.dot(X, T) 
    
    for j in range(p):
        if T[0, j] < 0:
            T[:, j] = -T[:, j]
    
    v = np.zeros((n, n))
    for i in range(n):
        Xi = np.delete(X, i, axis=0)  
        Si = np.cov(Xi, rowvar=False, bias=True)
        Ti, Di = eigsort(Si)
        
        for j in range(p):
            if Ti[0, j] < 0:
                Ti[:, j] = -Ti[:, j]
        
        Yi = np.dot(Xi, Ti)
        y = X[i, :] @ Ti 
        
        Yplusi = np.vstack([Yi[:i, :], y, Yi[i:, :]])
        
       #calculate euclidean distances 
        for k in range(n):
            v[k, i] = np.linalg.norm(Yplusi[k, :] - Y[k, :])
    
    v = np.sqrt(v) 
    #save summarize values in table to analysis
    Rad = np.zeros((n, 6))
    for i in range(n):
        Rad[i, 0] = np.min(v[i, :])
        Rad[i, 1] = np.max(v[i, :])
        Rad[i, 2] = np.quantile(v[i, :], 0.50)
        Rad[i, 3] = np.quantile(v[i, :], 0.75)
        Rad[i, 4] = np.quantile(v[i, :], 0.90)
        Rad[i, 5] = np.quantile(v[i, :], 0.95)
    
    rowsummary = np.vstack([np.mean(Rad, axis=0), np.std(Rad, axis=0), np.median(Rad, axis=0), np.median(np.abs(Rad - np.median(Rad, axis=0)), axis=0)])
    
    stats_labels = ['Mean', 'Standard Deviation', 'Median', 'Mean Absolute Deviation (MAD)']
    pca_labels = ['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4', 'PCA 5', 'PCA 6']
    
    rowsummary_df = pd.DataFrame(rowsummary.T, columns=stats_labels, index=pca_labels)
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 5))

    # Boxplot
    axes[0].boxplot(v)
    axes[0].set_title('Euclidean distance of each unit among configurations')

    # Barplot
    euclidean_distances = np.mean(v, axis=1)
    axes[1].bar(range(1, n + 1), euclidean_distances, color='orange', alpha=0.7)
    axes[1].set_xlabel('Observation Index')
    axes[1].set_ylabel('Euclidean Distance')
    axes[1].set_title('PCA Configuration Stability')
    axes[1].grid()

    # Show the plot
    plt.tight_layout()  # Adjust spacing
    plt.show()  
         
    return rowsummary_df


To perform the stability analysis using the leave-one-out method, we obtained 354 different outputs, which makes it challenging to interpret the results visually. Even though you can see that the values are very consistence across both graph.

The values in the table are quite consistent across the different PCAs, indicating that they all exhibit similar average Euclidean distances and very low standard deviations. This suggests that the principal components are stable, remaining unaffected by changes in the data. Consequently, this implies that the PCA captures the underlying structure of the data reliably, providing confidence in the robustness of the results. In general, this suggests that the PCA model is likely to generalize well to new data.

In [ ]:
PCAcovstability(data_standardized_numeric)

### Conclusion Part 1
 
In the first Part of this project work, an exploratory data analysis has been performed in support of a principal component analysis. The principal component analysis has been shown to deliver three principal components that explain the datas variance well and are stable when genaralizing to new data, while attributing intuitive characteristics to the new dimensions has shown to be challanging. Only when visualizing the categorical variables together with the data in the new dimensions, patterns and cluster emerge. To further understand the in this analysis discovered patterns will be the task of the second part of this work.

# Part 2: Multidimensional Scaling (MDS) and Clustering

In [ ]:
# Distance Matrix: mahalanobis

import numpy as np
from scipy.spatial.distance import mahalanobis

def maha_dist(data,numeric_variables):
    data_numeric = data[numeric_variables]

    # Step 1: Compute the covariance matrix and its inverse
    cov_matrix = np.cov(data_numeric.T)  # Transpose because rows are observations
    inv_cov_matrix = np.linalg.inv(cov_matrix)

    # Step 2: Create an empty distance matrix
    n = len(data_numeric)
    m_distance_mahalanobis = np.zeros((n, n))

    # Step 3: Compute pairwise Mahalanobis distances
    for i in range(n):
        for j in range(n):
            xi = data_numeric.iloc[i].values
            xj = data_numeric.iloc[j].values
            m_distance_mahalanobis[i, j] = mahalanobis(xi, xj, inv_cov_matrix)

    # Step 4: Convert distance matrix to a DataFrame for better readability
    df_distance_mahalanobis = pd.DataFrame(m_distance_mahalanobis, index=data_numeric.index, columns=data_numeric.index)
    return (m_distance_mahalanobis)

In [ ]:
maha_dist(data_standardized_numeric, numeric_variables)

In [ ]:
#import gower

# Compute Gower similarity between rows
similarity_matrix = gower.gower_matrix(data)
m_distance_gowers = 1 - similarity_matrix
df_distance_gowers = pd.DataFrame(m_distance_gowers, index=data.index, columns=data.index)
df_distance_gowers

## Multidimensional Scaling

- Distance Matrix -> distance measure
- pot force to fullfill euclidean property -> slides 5
- principal coordinates -> slide 6
- check euclidian property !!!
opt: 
- compare MDS configurations - split in groups (male/female) -> only when fulfilling equal geometric variability -> slide 10
- correlations: original variables vs first n principal coordinates heatmap
- group individuals via mds scatterplot with categorical colorcoding
- Gowers interpolation formula - > mds stability & variables partial influence on principal coordinates

- Interpretation of principal coordinates

## Usual K-means
 - Non-hierarchical
 - Minimize the WCSS (within-clusters sum of squeares)
 - Euclidean distance only for uncorrelated numerical variables.

 ### selection of number of clusters
 - Average silhouette
     Calulates the ratio between the difference in the distance of the selected group and the closest group and the max of the two of them.
     Near 1 is correctly classified, near to -1 should have been clasified in the neighborhood cluster. Near to 0, any of them.
     The greater the average silhouette the better the global quality of the clustering.
 - Elbow method 
     For large datasets. 

### PCA

In [ ]:
from sklearn.cluster import KMeans

X_pca = pca.fit_transform(data_pca)

# Elbow Method
wcss = []
for k in range(1, 11):  # Try k from 1 to 10
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_pca)
    wcss.append(kmeans.inertia_)  # inertia_ is the WCSS

# Plotting the Elbow Graph
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS')
plt.show()

# 2, 5 or 7 clusters

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
for k in range(2, 11):  # Silhouette score is undefined for k=1
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_pca)
    score = silhouette_score(X_pca, kmeans.labels_)
    silhouette_scores.append(score)

# Plotting the Silhouette Scores
plt.plot(range(2, 11), silhouette_scores)
plt.title('Silhouette Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.show()


In [ ]:
# Step 2: Perform K-Means clustering on the reduced data
kmeans = KMeans(n_clusters=2, random_state=14, init="random", n_init=30)
kmeans.fit(X_pca)

# Get the cluster labels
labels = kmeans.labels_

# Step 3: Visualize the results
#cmap='viridis'

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
scatterplot_pc(df_pca_merged,x_col="PC1",y_col = "PC2", hue_col=labels,
                ax = axs[0],expl_var = round(delta_explained_variance[0]+delta_explained_variance[1],3)*100)
scatterplot_pc(df_pca_merged,x_col="PC1",y_col = "PC3", hue_col=labels,
                ax = axs[1],expl_var = round(delta_explained_variance[0]+delta_explained_variance[2],3)*100)
scatterplot_pc(df_pca_merged,x_col="PC2",y_col = "PC3", hue_col=labels,
                ax = axs[2],expl_var = round(delta_explained_variance[1]+delta_explained_variance[2],3)*100)
plt.show()

# CHECKING THE CLUSTERS
# Get the inertia (WCSS)
inertia = kmeans.inertia_
print(f"Inertia (WCSS): {inertia}")

from sklearn.metrics import silhouette_score

# Assuming kmeans.labels_ contains the cluster labels
sil_score = silhouette_score(X_pca, kmeans.labels_)
print(f"Silhouette Score: {sil_score}")


In [ ]:
# Step 2: Perform K-Means clustering on the reduced data
kmeans5 = KMeans(n_clusters=5, random_state=14, init="random", n_init=30)
kmeans5.fit(X_pca)

# Get the cluster labels
labels = kmeans5.labels_

# Step 3: Visualize the results

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
scatterplot_pc(df_pca_merged,x_col="PC1",y_col = "PC2", hue_col=labels,
                ax = axs[0],expl_var = round(delta_explained_variance[0]+delta_explained_variance[1],3)*100)
scatterplot_pc(df_pca_merged,x_col="PC1",y_col = "PC3", hue_col=labels,
                ax = axs[1],expl_var = round(delta_explained_variance[0]+delta_explained_variance[2],3)*100)
scatterplot_pc(df_pca_merged,x_col="PC2",y_col = "PC3", hue_col=labels,
                ax = axs[2],expl_var = round(delta_explained_variance[1]+delta_explained_variance[2],3)*100)
plt.show()

# Get the inertia (WCSS)
inertia5 = kmeans5.inertia_
print(f"Inertia (WCSS): {inertia5}")

# Assuming kmeans.labels_ contains the cluster labels
sil_score = silhouette_score(X_pca, kmeans5.labels_)
print(f"Silhouette Score: {sil_score}")

In [ ]:
# Step 2: Perform K-Means clustering on the reduced data
kmeans7 = KMeans(n_clusters=7, random_state=14, init="random", n_init=30)
kmeans7.fit(X_pca)

# Get the cluster labels
labels = kmeans7.labels_

# Step 3: Visualize the results

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
scatterplot_pc(df_pca_merged,x_col="PC1",y_col = "PC2", hue_col=labels,
                ax = axs[0],expl_var = round(delta_explained_variance[0]+delta_explained_variance[1],3)*100)
scatterplot_pc(df_pca_merged,x_col="PC1",y_col = "PC3", hue_col=labels,
                ax = axs[1],expl_var = round(delta_explained_variance[0]+delta_explained_variance[2],3)*100)
scatterplot_pc(df_pca_merged,x_col="PC2",y_col = "PC3", hue_col=labels,
                ax = axs[2],expl_var = round(delta_explained_variance[1]+delta_explained_variance[2],3)*100)
plt.show()

# Get the inertia (WCSS)
inertia7 = kmeans7.inertia_
print(f"Inertia (WCSS): {inertia7}")

# Assuming kmeans.labels_ contains the cluster labels
sil_score = silhouette_score(X_pca, kmeans7.labels_)
print(f"Silhouette Score: {sil_score}")

### Original Data

In [ ]:
data_pca["pulse_pressure"] = data_pca['blood_pressure_systolic']-data_pca['blood_pressure_diastolic']
data_kmeans = data_pca.drop(['blood_pressure_diastolic', 'blood_pressure_systolic'], axis=1)
data_kmeans.head()

In [ ]:
# Elbow Method
wcss = []
for k in range(1, 11):  # Try k from 1 to 10
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_kmeans)
    wcss.append(kmeans.inertia_)  # inertia_ is the WCSS

# Plotting the Elbow Graph
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS')
plt.show()

# 2, 5 or 7 cluster
#_---------------------------------------------------------------------

silhouette_scores = []
for k in range(2, 11):  # Silhouette score is undefined for k=1
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_kmeans)
    score = silhouette_score(data_kmeans, kmeans.labels_)
    silhouette_scores.append(score)

# Plotting the Silhouette Scores
plt.plot(range(2, 11), silhouette_scores)
plt.title('Silhouette Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
# Step 2: Perform K-Means clustering on the reduced data
kmeans = KMeans(n_clusters=2, random_state=14, init="random", n_init=30)
kmeans.fit(data_kmeans)

# Add cluster labels to the DataFrame
data_kmeans['Cluster'] = kmeans.labels_

# Create pairplot to visualize the clusters
sns.pairplot(data_kmeans, hue='Cluster', palette='Dark2')
plt.title('Pairwise Plot of Clusters')
plt.show()

# CHECKING THE CLUSTERS
# Get the inertia (WCSS)
inertia = kmeans.inertia_
print(f"Inertia (WCSS): {inertia}")

sil_score = silhouette_score(data_kmeans, kmeans.labels_)
print(f"Silhouette Score: {sil_score}")


In [ ]:
# Step 2: Perform K-Means clustering on the reduced data
kmeans = KMeans(n_clusters=5, random_state=14, init="random", n_init=30)
kmeans.fit(data_kmeans)

# Add cluster labels to the DataFrame
data_kmeans['Cluster'] = kmeans.labels_

# Create pairplot to visualize the clusters
sns.pairplot(data_kmeans, hue='Cluster', palette='Dark2')
plt.title('Pairwise Plot of Clusters')
plt.show()

# CHECKING THE CLUSTERS
# Get the inertia (WCSS)
inertia = kmeans.inertia_
print(f"Inertia (WCSS): {inertia}")

sil_score = silhouette_score(data_kmeans, kmeans.labels_)
print(f"Silhouette Score: {sil_score}")
